# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\bengb\OneDrive\문서\GitHub\eeg_analysis


In [63]:
# Load some packages
import os
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_error_table

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0
cuda is available.


-----

## Load the configuration used during the train phase

In [4]:
model_name = 'luhprdie'
model_path = os.path.join('local/checkpoint_temp', model_name, 'last_checkpoint.pt')

ckpt = torch.load(model_path, map_location=device)
print(ckpt.keys())

dict_keys(['model_state', 'config', 'optimizer_state', 'scheduler_state'])


In [5]:
model_state = ckpt['model_state']
config = ckpt['config']
optimizer = ckpt['optimizer_state']
scheduler = ckpt['scheduler_state']

In [6]:
pprint.pprint(config, width=250)

{'EKG': 'X',
 '_target_': 'models.resnet_1d.ResNet1D',
 'activation': 'gelu',
 'age_mean': tensor([71.3768], device='cuda:0'),
 'age_std': tensor([9.7025], device='cuda:0'),
 'awgn': 0.01833870630857289,
 'awgn_age': 0.2504092552157154,
 'base_channels': 64,
 'base_lr': 0.001264911064067352,
 'block': 'bottleneck',
 'class_label_to_name': ['Normal', 'Abnormal'],
 'class_name_to_label': {'Abnormal': 1, 'Normal': 0},
 'conv_layers': [3, 4, 6, 3],
 'criterion': 'multi-bce',
 'crop_multiple': 4,
 'crop_timing_analysis': False,
 'cwd': 'C:\\Users\\bengb\\OneDrive\\문서\\GitHub\\eeg_analysis',
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/02_Curated_Data_220419/',
 'ddp': False,
 'device': device(type='cuda'),
 'draw_result': True,
 'dropout': 0.2699148150683074,
 'fc_stages': 4,
 'file_format': 'memmap',
 'groups': 32,
 'in_channels': 20,
 'input_norm': 'dataset',
 'iterations': 312500,
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'cosine_decay_with_warmu

-----

## Load the target model

In [7]:
# model = config['generator'](**config).to(device)
model = hydra.utils.instantiate(config).to(device)

if config.get('ddp', False):
    model_state_ddp = deepcopy(model_state)
    model_state = OrderedDict()
    for k, v in model_state_ddp.items():
        name = k[7:] # remove 'module.' of DataParallel/DistributedDataParallel
        model_state[name] = v
    model.load_state_dict(model_state)
else:
    model.load_state_dict(model_state)

-----

## Evaluate the model and analyze the performance by the crop timing

### Configurations

In [8]:
config = ckpt['config']

config.pop('cwd', 0)
config['ddp'] = False
config['crop_timing_analysis'] = True
config['eval'] = True
config['device'] = device

repeat = round(200 / config['crop_multiple'])
print(repeat)

50


### Build Dataset

In [9]:
train_loader, val_loader, test_loader, multicrop_test_loader = build_dataset_for_train(config, verbose=True)

transform: Compose(
    EegRandomCrop(crop_length=3000, length_limit=10000000, multiple=4, latency=2000, return_timing=True)
    EegDropChannels(drop_index=[19])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------

transform_multicrop: Compose(
    EegRandomCrop(crop_length=3000, length_limit=10000000, multiple=8, latency=2000, return_timing=True)
    EegDropChannels(drop_index=[19])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------


task config:
{'class_label_to_name': ['Normal', 'Abnormal'],
 'class_name_to_label': {'Abnormal': 1, 'Normal': 0},
 'task_description': 'Classification of [Normal] and [Abnormal] symptoms',
 'task_name': 'CAUEEG-task1 benchmark'}

 ---------------------------------------------------------------------------------------------------- 

train_dataset[0].keys():
dict_keys(['serial', 'age', 'symptom', 'class_name', 'class

### Train set

In [10]:
_ = check_accuracy_extended_debug(model, train_loader, 
                                  config['preprocess_test'], config, repeat=repeat)
train_acc = _[0] 
train_score = _[1]
train_target = _[2]
train_confusion = _[3]
train_error_table = _[4]
train_crop_timing = _[5]

print(train_acc)

KeyboardInterrupt: 

In [ ]:
draw_roc_curve(train_score, train_target, config['class_label_to_name'], use_wandb=False)
draw_confusion(train_confusion, config['class_label_to_name'], use_wandb=False)
draw_error_table(train_error_table, use_wandb=False, fig_size=(60.0, 4.0))

### Validation set

In [ ]:
_ = check_accuracy_extended_debug(model, val_loader, 
                                  config['preprocess_test'], config, repeat=repeat)
val_acc = _[0]
val_score = _[1]
val_target = _[2]
val_confusion = _[3]
val_error_table = _[4]
val_crop_timing = _[5]

print(val_acc)

In [ ]:
draw_roc_curve(val_score, val_target, config['class_label_to_name'], use_wandb=False)
draw_confusion(val_confusion, config['class_label_to_name'], use_wandb=False)
draw_error_table(val_error_table, use_wandb=False)

### Test set

In [11]:
_ = check_accuracy_extended_debug(model, test_loader, 
                                  config['preprocess_test'], config, repeat=repeat)
test_acc = _[0]
test_score = _[1]
test_target = _[2]
test_confusion = _[3]
test_error_table = _[4]
test_crop_timing = _[5]

print(test_acc)

70.5431654676259


In [76]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=False, labelbottom=True)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [67]:
class_label_to_name = config['class_label_to_name']
C = len(class_label_to_name)

ax = fig.add_subplot(2, 1, 1)

im, cbar = heatmap(test_confusion, class_label_to_name, class_label_to_name, ax=ax,
                   cmap="jet", cbarlabel="test data")
texts = annotate_heatmap(im, valfmt="{x:d}")

fig.tight_layout()
plt.show()

ax = fig.add_subplot(2, 1, 2)
from train.evaluate import calculate_class_wise_metrics
class_wise_metrics = calculate_class_wise_metrics(confusion)
ax.imshow(np.zeros((C, len(class_wise_metrics) + 1)), alpha=0.8)

ax.set_xticks(np.arange(len(class_wise_metrics) + 1))
ax.set_yticks(np.arange(C))
ax.set_xticklabels([*class_wise_metrics.keys()] + ['Accuracy'])
ax.set_yticklabels(class_label_to_name)

for c in range(len(class_wise_metrics)):
    for r in range(C):
        metric_name = [*class_wise_metrics.keys()][c]
        ax.text(c, r, f"{class_wise_metrics[metric_name][r] * 100:.2f}%",
                ha="center", va="center", color='k')
        
ax.text(len(class_wise_metrics), 0, f"{np.trace(confusion) / np.sum(confusion) * 100:.2f}%",
        ha="center", va="center", color='k')

ax.set_title('Class-wise metrics')
ax.set_ylabel('Class')
ax.set_xlabel('Metric')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# draw
fig.savefig(os.path.join(output_folder, 'confusion_temp.pdf'), transparent=True)

plt.show()
fig.clear()
plt.close(fig)

In [ ]:
draw_roc_curve(test_score, test_target, config['class_label_to_name'], use_wandb=False)
draw_confusion(test_confusion, config['class_label_to_name'], use_wandb=False)
draw_error_table(test_error_table, use_wandb=False)

In [ ]:
_ = check_accuracy_extended(model, test_loader, 
                            config['preprocess_test'], config, repeat=repeat)
test_acc = _[0]
test_score = _[1]
test_target = _[2]
test_confusion = _[3]
test_error_table = _[4]

print(test_acc)

### Test set (with test-time augmentation)

In [ ]:
_ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                      config['preprocess_test'], config, repeat=repeat)
multi_test_acc = _[0]
multi_test_score = _[1]
multi_test_target = _[2]
multi_test_confusion = _[3]
multi_test_error_table = _[4]

print(multi_test_acc)

In [ ]:
draw_roc_curve(multi_test_score, multi_test_target, config['class_label_to_name'], use_wandb=False)
draw_confusion(multi_test_confusion, config['class_label_to_name'], use_wandb=False)
draw_error_table(multi_test_error_table, use_wandb=False)

### Examine whether the model has a bias related to the cropping starting time

In [ ]:
train_pos = []
train_neg = []

for k, v in train_crop_timing.items():
    for i in range(v.get('correct', 0)):
        train_pos.append(k)
    for i in range(v.get('incorrect', 0)):
        train_neg.append(k)

##

val_pos = []
val_neg = []

for k, v in val_crop_timing.items():
    for i in range(v.get('correct', 0)):
        val_pos.append(k)
    for i in range(v.get('incorrect', 0)):
        val_neg.append(k)

##

test_pos = []
test_neg = []

for k, v in test_crop_timing.items():
    for i in range(v.get('correct', 0)):
        test_pos.append(k)
    for i in range(v.get('incorrect', 0)):
        test_neg.append(k)

In [ ]:
fig = plt.figure(num=1, clear=True, figsize=(16, 10), constrained_layout=True)
n_bins = 50
density = False

ax = fig.add_subplot(3, 1, 1)
ax.hist(train_pos, bins=n_bins, color='g', alpha=0.7, density=density)
ax.hist(train_neg, bins=n_bins, color='r', alpha=0.7, density=density)
ax.set_title(f'Training Set: True/Neg Historgram by Random Crop Time')
ax.set_xlabel('Random Crop Time')
ax.set_ylabel('Count')

ax = fig.add_subplot(3, 1, 2)
ax.hist(val_pos, bins=n_bins, color='g', alpha=0.7, density=density)
ax.hist(val_neg, bins=n_bins, color='r', alpha=0.7, density=density)
ax.set_title(f'Validation Set: True/Neg Historgram by Random Crop Time')
ax.set_xlabel('Random Crop Time')
ax.set_ylabel('Count')

ax = fig.add_subplot(3, 1, 3)
ax.hist(test_pos, bins=n_bins, color='g', alpha=0.7, density=density)
ax.hist(test_neg, bins=n_bins, color='r', alpha=0.7, density=density)
ax.set_title(f'Test Set: True/Neg Historgram by Random Crop Time')
ax.set_xlabel('Random Crop Time')
ax.set_ylabel('Count')

plt.show()
fig.clear()
plt.close(fig)

-----

## Evaluate the model with length limit

### Configurations

In [ ]:
config = ckpt['config']

config['crop_timing_analysis'] = False
config['signal_length_limit'] = 200 * 60 * 7  # 7 minutes
config['eval'] = True
config['device'] = device

repeat = round(50 / config['crop_multiple'])

train_loader, val_loader, test_loader, multi_test_loader = build_dataset_for_train(config, verbose=False)

In [ ]:
# train
train_acc = check_accuracy(model, train_loader, 
                           config['preprocess_test'], config, repeat=repeat)
val_acc = check_accuracy(model, val_loader, 
                         config['preprocess_test'], config, repeat=repeat)
test_acc = check_accuracy(model, test_loader, 
                          config['preprocess_test'], config, repeat=repeat)
multi_test_acc = check_accuracy_multicrop(model, multi_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)

print(train_acc, val_acc, test_acc, multi_test_acc)